# Loading and Saving Data

to/from csv, jld, mat and xls files.

# Load Packages

none here. Instead, the packages are loaded in the respective cells below. This allows you to run parts of this notebook without having to install all packages.

All files are written to and loaded from the subfolder "Results".

In [1]:
include("printmat.jl")            #a function for prettier matrix printing

if !isdir("Results")
    error("create the subfolder Results before running this program")
end

# Saving and Loading a csv File

This functionality is built in to Julia. No package needed.

The simplest commands are
```
writedlm(FileName,matrix)
x = readdlm(FileName)
```

Extra arguments control which delimiters (',' perhaps) in the file, the type of data (Float, Int, etc), and whether the file starts with a header line with variable names. 

For instance, if you want to specify that the delimter is comma and also disregard the first 3 lines (perhaps because the contain variables names etc), then use

```
x = readdlm(FileName,',',skipstart=3)
```

In [2]:
A = copy(reshape(1:20,5,4))     #to be on the safe side: only try to save
                                #"independent" arrays, not reshapes or views 

writedlm("Results/NewCsvFile.csv",A,',')  
println("NewCsvFile.csv has been created in the subfolder Results. Check it out.")

NewCsvFile.csv has been created in the subfolder Results. Check it out.


In [3]:
A2 = readdlm("Results/NewCsvFile.csv",',',Int)  #try Float64 instead of Int

println("\nA (in memory):")
printmat(A)
println("\nA2 (loaded from csv file which we just created and then reloaded):")
printmat(A2)


A (in memory):
         1         6        11        16
         2         7        12        17
         3         8        13        18
         4         9        14        19
         5        10        15        20


A2 (loaded from csv file which we just created and then reloaded):
         1         6        11        16
         2         7        12        17
         3         8        13        18
         4         9        14        19
         5        10        15        20



# Saving and Loading a jld File

jld files can store very different types of data: integers, floats, strings, dictionaries, etc. They are also quick to load (you'll notice the difference to csv files once you have a few thousand data points).

The basic syntax is 
```
save(FileName,"MatrixName",matrix)
x = load(FileName,"MatrixName")  
```

The package JLD is at https://github.com/JuliaLang/JLD.jl. 

There is also a new package with similar functionality (with the advantage that is does not depend on external libraries), see https://github.com/simonster/JLD2.jl.

In [4]:
using JLD       

B   = copy(reshape(1:20,5,4))
B27 = 1

save("Results/NewJldFile.jld","B",B,"B27",B27)
println("NewJldFile.jld has been created in the subfolder Results")

NewJldFile.jld has been created in the subfolder Results


In [5]:
B2 = load("Results/NewJldFile.jld","B")          #reload the data from the jld file
println("\nB from jld file is")
printmat(B2)
                                                 #alternative approach
xx = load("Results/NewJldFile.jld")                      #load all variables into dictionary
println("\nAll variables in the JLD file: ",keys(xx))
B3 = xx["B"]
println("\nB from jld file is (2nd way of loading)")
printmat(B3)


B from jld file is
         1         6        11        16
         2         7        12        17
         3         8        13        18
         4         9        14        19
         5        10        15        20


All variables in the JLD file: String["B", "B27"]

B from jld file is (2nd way of loading)
         1         6        11        16
         2         7        12        17
         3         8        13        18
         4         9        14        19
         5        10        15        20



# (extra) Saving and Loading a Matlab mat File

The MAT package allows you to load/save (Matlab) mat files. See https://github.com/simonster/MAT.jl.

In [6]:
using MAT     

C   = copy(reshape(1:20,5,4))
C27 = 1

fh = matopen("Results/NewMatFile.mat","w")               #create a mat file
write(fh,"C",C)
write(fh,"C27",C27)
close(fh)

println("\nNewMatFile.mat has been created in the subfolder Results")


NewMatFile.mat has been created in the subfolder Results


In [7]:
fh = matopen("Results/NewMatFile.mat")           #open the mat file
println("\nVariables in mat file: ",names(fh))
C2 = read(fh,"C")                                #read variable C
close(fh)                                        #close the mat file
println("\nC from mat file is ")
printmat(C2)
                                                 #alternative approach
xx = matread("Results/NewMatFile.mat")                   #read whole mat file into Dict xx
C3 = xx["C"]
println("\nC from mat file is (2nd way of loading) ")
printmat(C3)


Variables in mat file: String["C", "C27"]

C from mat file is 
         1         6        11        16
         2         7        12        17
         3         8        13        18
         4         9        14        19
         5        10        15        20


C from mat file is (2nd way of loading) 
         1         6        11        16
         2         7        12        17
         3         8        13        18
         4         9        14        19
         5        10        15        20



# (extra) Loading csv and Fixing Missing Values

In [8]:
x = readdlm("Data/loadCsvTsT_Data.csv",',',skipstart=1)   #load the data

println("\nx")
printmat(x)


x
      1861    10.094     9.998          
      1862    10.128     9.994     0.037
      1863    10.156    10.044     0.092
      1864              10.061     0.098
      1865    10.197    10.076     0.078
      1866    10.234    10.064     0.061
      1867    10.242     10.06     0.025
      1868    10.186    10.057    -0.002
      1869    10.235    10.068     0.107
      1870    10.343     10.16     0.071
      1871    10.379    10.188     0.022
      1872    10.438    10.256    -0.022
      1873    10.494    10.315     -0.06
      1874    10.505    10.337     0.005
      1875    10.485    10.305     0.074



In [9]:
include("readdlmFixPs.jl")                       #function for fixing missing values, a real hack
x2 = readdlmFixPs(x)                             #replace all missing values with NaN
                                                 #convert to Float matrix
println("\nafter fix")
printmat(x2)


after fix
  1861.000    10.094     9.998       NaN
  1862.000    10.128     9.994     0.037
  1863.000    10.156    10.044     0.092
  1864.000       NaN    10.061     0.098
  1865.000    10.197    10.076     0.078
  1866.000    10.234    10.064     0.061
  1867.000    10.242    10.060     0.025
  1868.000    10.186    10.057    -0.002
  1869.000    10.235    10.068     0.107
  1870.000    10.343    10.160     0.071
  1871.000    10.379    10.188     0.022
  1872.000    10.438    10.256    -0.022
  1873.000    10.494    10.315    -0.060
  1874.000    10.505    10.337     0.005
  1875.000    10.485    10.305     0.074



# (extra) Loading xlsfile


You need python and python's xlrd libarary for this to work. 

See https://github.com/davidanthoff/ExcelReaders.jl

In [10]:
using ExcelReaders


println("\n------------Approach 1: readxl-------------------")

data1 = readxl("Data/readXlsTsT_Data.xlsx","Data!B2:C11")       #reading using range
x1    = convert(Array{Float64},data1)            #convert from DataArray to traditional matrix
println("\nNumeric part after conversion:")
printmat(x1)


------------Approach 1: readxl-------------------

Numeric part after conversion:
    16.660  -999.990
    16.850  -999.990
    16.930  -999.990
    16.980  -999.990
    17.080  -999.990
    17.030     7.000
    17.090     8.000
    16.760  -999.990
    16.670  -999.990
    16.720  -999.990



In [11]:
println("\n------------Approach 2: readxlsheet--------------")

data2 = readxlsheet("Data/readXlsTsT_Data.xlsx","Data",skipstartrows=1)  #reading all columns
x2    = convert(Array{Float64},data2[:,2:end])
println("\nNumeric part after conversion: ")
printmat(x2)

x2[x2 .== -999.99] = NaN                     #convert -999.99 to NaNs
println("\nNumeric part after changing -999.99 to NaN:")
printmat(x2)


------------Approach 2: readxlsheet--------------

Numeric part after conversion: 
    16.660  -999.990
    16.850  -999.990
    16.930  -999.990
    16.980  -999.990
    17.080  -999.990
    17.030     7.000
    17.090     8.000
    16.760  -999.990
    16.670  -999.990
    16.720  -999.990


Numeric part after changing -999.99 to NaN:
    16.660       NaN
    16.850       NaN
    16.930       NaN
    16.980       NaN
    17.080       NaN
    17.030     7.000
    17.090     8.000
    16.760       NaN
    16.670       NaN
    16.720       NaN

